### Задание 2. Computational Graph

$2*a*b + x - 3*b*a -x$

In [61]:
class Operation():    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes 
        self.output_nodes = [] 
        for node in input_nodes:
            node.output_nodes.append(self)
        _default_graph.operations.append(self)
  
    def compute(self):
        pass

In [62]:
class add(Operation):
    
    def __init__(self, x, y):
        super().__init__([x, y])

    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [63]:
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var

In [64]:
class subtract(Operation):
     
    def __init__(self, d, e):
        
        super().__init__([d, e])
    
    def compute(self, d_var, e_var):
         
        self.inputs = [d_var, e_var]
        return d_var - e_var

In [65]:
class Placeholder():
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [66]:
class Variable():
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [67]:
class Graph():
    def __init__(self):
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [68]:
g = Graph()
g.set_as_default()

In [69]:
x = Placeholder()
a = Placeholder()
b = Placeholder()

In [70]:
c = Variable(2)
d = Variable(3)

In [71]:
h = multiply(a,b)
p = multiply(c,h)
i = multiply(d,h)

k = add(p,x)
l = subtract(k,i)
m = subtract(l,x)

In [72]:
def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [73]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                node.output = node.value
                
            else:                
                node.inputs = [input_node.output for input_node in node.input_nodes]
                 
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        return operation.output

In [74]:
sess = Session()

In [75]:
test = [[1,4,2], [-3,2,0], [12,7,-3], [5,21,14], [-2,-4,-9]]
print('Default tests: ', *test)
n = input("Do you want to add additional tests? y/n ")
if n == 'y':
    new = input('Insert new test (use commas): ')
    new = new.split(',')
    test.append([int(num) for num in new])

Default tests:  [1, 4, 2] [-3, 2, 0] [12, 7, -3] [5, 21, 14] [-2, -4, -9]
Do you want to add additional tests? y/n n


In [76]:
for num, test in enumerate(test):
    print('Test #' + str(num+1))
    print(sess.run(operation=m,feed_dict={x:test[0], a:test[1], b:test[2]}))

Test #1
-8
Test #2
0
Test #3
21
Test #4
-294
Test #5
-36
